In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import earthpy as et
from shapely.geometry import Point, Polygon

In [12]:
#This functions checks if the coordinates of the radiation data are located inside the
#selected polygon.
def evaluate_point(polygon,global_radiation):
    points_inside_polygon = []
    for i in range(len(global_radiation)):
        point = Point(global_radiation.iloc[i,1],global_radiation.iloc[i,0])
        if point.within(polygon) == True:
            points_inside_polygon.append(point)
    
    return points_inside_polygon

In [3]:
#This function gets the average radiation for each state.
def average_radiation(points,global_radiation):
    radiation_points = []
    average_radiation =[]
    for i in range(len(points)):
        for point in points[i]:
            LON = point.x
            LAT = point.y
            row = global_radiation.loc[(global_radiation['LON'] == LON) & (global_radiation['LAT'] == LAT)]
            radiation = row["ANN"]
            radiation_points.append(float(radiation))
        average_radiation.append(sum(radiation_points) / len(radiation_points))
    
    return average_radiation
            

In [4]:
#This function computes the global radiation per month per state
def average_radiation_month(points,global_radiation):
    radiation_points = []
    average_radiation =[]
    radiation = []
    for i in range(len(points)):
        for point in points[i]:
            LON = point.x
            LAT = point.y
            row = global_radiation.loc[(global_radiation['LON'] == LON) & (global_radiation['LAT'] == LAT)]
            radiation = [row.iloc[0,rad] for rad in range(3,16)]         
            radiation_points.append((radiation))
            radiation_array = np.array(radiation_points)
        average_radiation.append(np.average(radiation_array,axis=0))
    
    return average_radiation

In [5]:
#Read the shp files.
mapa = gpd.read_file('México_Estados.shp')

In [7]:
#Read NASA's file that contains the global average radiations
global_radiation = pd.read_csv("POWER_Global_Climatology_8fd627aa_mexico.csv")

In [13]:
#Apply the average radiation per month funtion to all the polygons inside the "mapa" geopandas dataframe
radiation = []
state_list = []
for state in range(len(mapa)):
    state_list.append(str(mapa["ESTADO"][state]))
    points_inside_polygon = evaluate_point(mapa["geometry"][state],global_radiation)
    radiation.append(points_inside_polygon)
avg_radiations = average_radiation_month(radiation,global_radiation)

In [23]:
#Covert the list of lists with the average radiations to a dataframe and add the month name to the columns.
months = ["Ene","Feb","Mar","Abr","May","Jun","Jul","Ago","Sep","Oct","Nov","Dic","Anual"]
df_rad_avg_month = pd.DataFrame(avg_radiations,index=state_list,columns=months)

In [24]:
df_rad_avg_month

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
Baja California,3.529310,4.352069,5.547241,6.668621,7.005862,6.980690,6.688276,6.209655,5.608621,4.633793,3.828966,3.265517,5.360690
Baja California Sur,3.751964,4.568929,5.766250,6.735893,7.124107,7.083929,6.740714,6.254821,5.653750,4.833036,4.031250,3.463393,5.500357
Nayarit,3.870769,4.710462,5.909846,6.825846,7.206000,7.050308,6.645538,6.202000,5.615692,4.919538,4.166615,3.584769,5.558615
Jalisco,4.146882,5.028280,6.204086,6.958710,7.202366,6.811290,6.378495,6.035054,5.497419,5.035806,4.455591,3.880215,5.635484
Aguascalientes,4.163438,5.047708,6.221875,6.963646,7.199792,6.798229,6.368333,6.033438,5.496563,5.048437,4.476771,3.900938,5.642500
Guanajuato,4.217290,5.109533,6.263738,6.957290,7.155514,6.742804,6.329533,6.025514,5.480187,5.063832,4.530280,3.962150,5.652523
Querétaro,4.230360,5.125766,6.271892,6.950090,7.137477,6.724144,6.316757,6.023063,5.470631,5.062342,4.540721,3.975135,5.651712
Hidalgo,4.242458,5.137797,6.265847,6.916356,7.092203,6.675085,6.283983,6.004831,5.440254,5.044153,4.543305,3.988644,5.635763
Michoacán,4.333529,5.240588,6.353971,6.943015,7.043750,6.556250,6.183382,5.927132,5.379191,5.044191,4.614265,4.080294,5.641029
México,4.364236,5.271528,6.368681,6.926528,7.007222,6.512569,6.160486,5.911111,5.364444,5.045556,4.633125,4.110069,5.638958


In [26]:
#Read the csv file with the area of each state
state_area = pd.read_csv('Areas_Estados.csv')

In [30]:
#Compute the total radiation for each state by multiplying the average radiation by the area of each state.
total_radiation = df_rad_prom_mes.copy()
for estado in range(len(areas_estados)):
    for rad in range(13):
            radiacion_total.iloc[estado,rad] = radiacion_total.iloc[estado,rad] * areas_estados.iloc[estado,1]

radiacion_total.head()

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
Baja California,15.359802,18.940504,105.067549,126.307039,132.694561,132.217782,126.679318,117.614000,106.230104,87.766378,72.522538,61.850543,101.534165
Baja California Sur,17.142457,20.875108,120.371093,140.612493,148.716508,147.877776,140.713141,130.570075,118.022643,100.890144,84.152780,72.298701,114.820551
Nayarit,18.233110,22.188448,131.130314,151.454932,159.889956,156.435385,147.454184,137.612754,124.603496,109.156923,92.450728,79.540465,123.337048
Jalisco,20.851681,25.283595,156.861601,175.941200,182.101698,172.213909,161.271278,152.587858,138.994527,127.323293,112.653370,98.105788,142.485294
Aguascalientes,21.015818,25.479359,158.529389,177.429235,183.446080,173.214524,162.261054,153.728123,140.048892,128.630954,114.065253,99.393389,143.767286


In [229]:
dias_mes = {'Ene': 31, 'Feb': 28, 'Mar': 31, 'Abr': 30, 'May': 31, 'Jun': 30, 'Jul': 31, 'Ago': 31, 'Sep': 30, 'Oct': 31, 'Nov': 30, 'Dic': 31}

In [240]:
dias_df = radiacion_total.copy()
dias_df.head()

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
index,,,,,,,,,,,,,
Aguascalientes,23381.865000,28347.930000,34942.050000,39107.835000,40434.030000,38178.855000,35764.560000,33883.785000,30868.695000,28352.025000,25141.545000,21907.665000,31688.280000
Baja California,252169.224138,310955.327586,396350.396552,476472.948276,500568.844828,498770.275862,477877.310345,443679.862069,400735.948276,331084.517241,273579.586207,233321.206897,383021.275862
Baja California Sur,277303.928393,337684.941786,426177.771250,497843.105179,526535.634821,523566.076786,498199.452143,462287.596964,417863.008750,357204.836607,297945.656250,255975.902679,406525.896071
Campeche,251957.764358,303129.356313,363148.673296,394677.930168,398189.391117,370258.337598,353950.765978,340759.495475,310152.278212,289806.365866,266035.734972,236851.735642,323198.978045
Chiapas,329498.503591,390344.173490,459465.622047,492374.388725,486305.320369,447170.038893,440043.127248,425742.562047,385428.892349,366087.580872,342679.722987,311810.379765,406371.729362


In [258]:
dias_df["Anual"] = dias_df["Anual"] * 365

In [259]:
dias_df.head()

,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
index,,,,,,,,,,,,,
Aguascalientes,7.248378e+05,7.937420e+05,1.083204e+06,1.173235e+06,1.253455e+06,1.145366e+06,1.108701e+06,1.050397e+06,9.260609e+05,8.789128e+05,7.542463e+05,6.791376e+05,1.156622e+07
Baja California,7.817246e+06,8.706749e+06,1.228686e+07,1.429419e+07,1.551763e+07,1.496311e+07,1.481420e+07,1.375408e+07,1.202208e+07,1.026362e+07,8.207388e+06,7.232957e+06,1.398028e+08
Baja California Sur,8.596422e+06,9.455178e+06,1.321151e+07,1.493529e+07,1.632260e+07,1.570698e+07,1.544418e+07,1.433092e+07,1.253589e+07,1.107335e+07,8.938370e+06,7.935253e+06,1.483820e+08
Campeche,7.810691e+06,8.487622e+06,1.125761e+07,1.184034e+07,1.234387e+07,1.110775e+07,1.097247e+07,1.056354e+07,9.304568e+06,8.983997e+06,7.981072e+06,7.342404e+06,1.179676e+08
Chiapas,1.021445e+07,1.092964e+07,1.424343e+07,1.477123e+07,1.507546e+07,1.341510e+07,1.364134e+07,1.319802e+07,1.156287e+07,1.134872e+07,1.028039e+07,9.666122e+06,1.483257e+08


In [241]:
areas_estados

,Estado,Superficie km2
0,Aguascalientes,5616
1,Baja California,71450
2,Baja California Sur,73909
3,Campeche,57507
4,Chiapas,73311
5,Chihuahua,247455
6,Ciudad de México,1495
7,Coahuila de Zaragoza,151562
8,Colima,5627
9,Durango,123317


In [260]:
consumos_mensuales = pd.read_csv('gqotxvechn_IIIA3C06_16072021_00_26.csv')

In [264]:
consumos_mensuales.head()

,Estado,Ene/2000,Feb/2000,Mar/2000,Abr/2000,May/2000,Jun/2000,Jul/2000,Ago/2000,Sep/2000,...,Jun/2020,Jul/2020,Ago/2020,Sep/2020,Oct/2020,Nov/2020,Dic/2020,Ene/2021,Feb/2021,Mar/2021
0,Total,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,97107700,0,0,0
1,Aguascalientes,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1302600,0,0,0
2,Baja California,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,4469800,0,0,0
3,Baja California Sur,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1010600,0,0,0
4,Campeche,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,650000,0,0,0


In [267]:
dias_df = dias_df.reset_index()
dias_df.head()

,index,Ene,Feb,Mar,Abr,May,Jun,Jul,Ago,Sep,Oct,Nov,Dic,Anual
0,Aguascalientes,7.248378e+05,7.937420e+05,1.083204e+06,1.173235e+06,1.253455e+06,1.145366e+06,1.108701e+06,1.050397e+06,9.260609e+05,8.789128e+05,7.542463e+05,6.791376e+05,1.156622e+07
1,Baja California,7.817246e+06,8.706749e+06,1.228686e+07,1.429419e+07,1.551763e+07,1.496311e+07,1.481420e+07,1.375408e+07,1.202208e+07,1.026362e+07,8.207388e+06,7.232957e+06,1.398028e+08
2,Baja California Sur,8.596422e+06,9.455178e+06,1.321151e+07,1.493529e+07,1.632260e+07,1.570698e+07,1.544418e+07,1.433092e+07,1.253589e+07,1.107335e+07,8.938370e+06,7.935253e+06,1.483820e+08
3,Campeche,7.810691e+06,8.487622e+06,1.125761e+07,1.184034e+07,1.234387e+07,1.110775e+07,1.097247e+07,1.056354e+07,9.304568e+06,8.983997e+06,7.981072e+06,7.342404e+06,1.179676e+08
4,Chiapas,1.021445e+07,1.092964e+07,1.424343e+07,1.477123e+07,1.507546e+07,1.341510e+07,1.364134e+07,1.319802e+07,1.156287e+07,1.134872e+07,1.028039e+07,9.666122e+06,1.483257e+08


In [270]:
dias_df.to_excel('Radiacion_por_estado.xlsx')